## Access this Notebook
You can launch this notebook in the US GHG Center JupyterHub by clicking the link below. If you are a new user, you should first sign up for the hub by filling out this [**request form**](https://docs.google.com/forms/d/e/1FAIpQLSdai8otCdrVQzJgev8mjDhzKyCg7jcrB3UeTXNHoCiaMKrkaQ/viewform) and providing the required information. 


Access the [**EMIT Methane Point Source Plume Complexes**](https://us-ghg-center.github.io/ghgc-docs/user_data_notebooks/emit-ch4plume-v1_User_Notebook.html) notebook in the US GHG Center JupyterHub. 

## Table of Contents
- [Data Summary and Application](#data-summary-and-application)
- [Approach](#approach)
- [About the Data](#about-the-data)
- [Install the Required Libraries](#install-the-required-libraries)
- [Query the STAC API](#query-the-stac-api)
- [Visual Comparison Across Time Periods](#visual-comparison-across-time-periods)
- [Map Out Selected Tiles](#map-out-selected-tiles)
- [Summary](#summary)

## Data Summary and Application
- **Spatial coverage**: 52°N to 52°S latitude within target mask
- **Spatial resolution**: 60 m
- **Temporal extent**: August 1, 2022 - Ongoing
- **Temporal resolution**: Variable (based on ISS orbit, solar illumination, and target mask)
- **Unit**: Parts per million meter (ppm-m)
- **Utility**: Methane Emissions, Plume Detection, Climate Monitoring 

For more, visit the [EMIT Methane Point Source Plume Complexes](https://earth.gov/ghgcenter/data-catalog/emit-ch4plume-v1) data overview page.  

## Approach

1. Identify available dates and temporal frequency of observations for the given collection using the GHGC API `/stac` endpoint. The collection processed in this notebook is the Earth Surface Mineral Dust Source Investigation (EMIT) methane emission plumes data product.
2. Pass the STAC item into the raster API `/collections/{collection_id}/items/{item_id}/tilejson.json` endpoint.
3. Using `folium.Map`, visualize the plumes.
4. After the visualization, perform zonal statistics for a given polygon.

   

## About the Data
#### EMIT Methane Point Source Plume Complexes
The Earth Surface Mineral Dust Source Investigation (EMIT) instrument builds upon NASA’s long history of developing advanced imaging spectrometers for new science and applications. EMIT launched to the International Space Station (ISS) on July 14, 2022. The data shows high-confidence research grade methane plumes from point source emitters - updated as they are identified - in keeping with Jet Propulsion Laboratory (JPL) Open Science and Open Data policy. 

Large methane emissions, typically referred to as point source emissions, represent a significant proportion of total methane emissions from the production, transport, and processing of oil and natural gas, landfills, and other sources. By measuring the spectral fingerprint of methane, EMIT can map areas of high methane concentration over background levels in the atmosphere, identifying plume complexes, and estimating the methane enhancements.

For more information regarding this dataset, please visit the [EMIT Methane Point Source Plume Complexes](https://earth.gov/ghgcenter/data-catalog/emit-ch4plume-v1) data overview page.

## Terminology
Navigating data via the GHGC API, you will encounter terminology that is different from browsing in a typical filesystem. We'll define some terms here which are used throughout this notebook.
- `catalog`:    All datasets available at the `/stac` endpoint
- `collection`: A specific dataset, e.g. EMIT Methane Point Source Plume Complexes
- `item`:       One granule in the dataset, e.g. one file of EMIT data
- `asset`:      A variable available within the granule, e.g. CH4 plume emissions
- `STAC API`:   **Sp**atio**T**emporal **A**sset **C**atalogs - Endpoint for fetching metadata about available datasets
- `Raster API`: Endpoint for fetching data itself, for imagery and statistics

# Install the Required Libraries
Required libraries are pre-installed on the GHG Center Hub, except the `tabulate` and `seaborn` libraries. If you need to run this notebook elsewhere, please install the libraries by running the following command line:

%pip install requests folium rasterstats pystac_client pandas matplotlib --quiet

In [1]:
# Import the following libraries
# For fetching from the Raster API
import requests
# For making maps
import folium
import folium.plugins
from folium import Map, TileLayer
# For talking to the STAC API
from pystac_client import Client
# For working with data
import pandas as pd
# For making time series
import matplotlib.pyplot as plt
# For formatting date/time data
import datetime
# Custom functions for working with GHGC data via the API
import ghgc_utils

# Query the STAC API

**STAC API Collection Names**

Now, you must fetch the dataset from the [**STAC API**](https://earth.gov/ghgcenter/api/stac/) by defining its associated STAC API collection ID as a variable. 
The collection ID, also known as the **collection name**, for the EMIT Methane Point Source Plume Complexes dataset is [**emit-ch4plume-v1**](https://earth.gov/ghgcenter/api/stac/collections/emit-ch4plume-v1)*

**You can find the collection name of any dataset on the GHGC data portal by navigating to the dataset landing page within the data catalog. The collection name is the last portion of the dataset landing page's URL, and is also listed in the pop-up box after clicking "ACCESS DATA."*

In [2]:
# Provide the STAC and RASTER API endpoints
# The endpoint is referring to a location within the API that executes a request on a data collection nesting on the server.

# The STAC API is a catalog of all the existing data collections that are stored in the GHG Center.
STAC_API_URL = "https://earth.gov/ghgcenter/api/stac"

# The RASTER API is used to fetch collections for visualization
RASTER_API_URL = "https://earth.gov/ghgcenter/api/raster"

# The collection name is used to fetch the dataset from the STAC API. First, we define the collection name as a variable
collection_name = "emit-ch4plume-v1"

In [3]:
# Fetch the collection from the STAC API using the appropriate endpoint
# The 'pystac_client' library allows a HTTP request possible
catalog = Client.open(STAC_API_URL)
collection = catalog.get_collection(collection_name)

# Print the properties of the collection to the console
collection

<CollectionClient id=emit-ch4plume-v1>

Examining the contents of our `collection` under the `temporal` variable, we note that data is available from August 2022 to May 2023. By looking at the `dashboard: time density`, we can see that observations are conducted daily and non-periodically (i.e., there are plumes emissions for multiple places on the same dates).

In [4]:
items = list(collection.get_items())  # Convert the iterator to a list
print(f"Found {len(items)} items")

Found 1574 items


In [5]:
# The search function lets you search for items within a specific date/time range
search = catalog.search(
    collections=collection_name,
    datetime=['2023-08-24T00:00:00Z','2023-08-26T00:00:00Z']
)
# Take a look at the items we found
print(f"# items in date range: {len(search.item_collection())}")
for item in search.item_collection():
    print(item)

# items in date range: 24
<Item id=EMIT_L2B_CH4PLM_001_20230825T170609_001111>
<Item id=EMIT_L2B_CH4PLM_001_20230825T170609_001109>
<Item id=EMIT_L2B_CH4PLM_001_20230825T170557_001108>
<Item id=EMIT_L2B_CH4PLM_001_20230825T075005_001115>
<Item id=EMIT_L2B_CH4PLM_001_20230825T075005_001114>
<Item id=EMIT_L2B_CH4PLM_001_20230825T074743_001112>
<Item id=EMIT_L2B_CH4PLM_001_20230825T061313_001116>
<Item id=EMIT_L2B_CH4PLM_001_20230824T175401_001039>
<Item id=EMIT_L2B_CH4PLM_001_20230824T175337_001038>
<Item id=EMIT_L2B_CH4PLM_001_20230824T175337_001037>
<Item id=EMIT_L2B_CH4PLM_001_20230824T175337_001036>
<Item id=EMIT_L2B_CH4PLM_001_20230824T083931_001044>
<Item id=EMIT_L2B_CH4PLM_001_20230824T083907_001045>
<Item id=EMIT_L2B_CH4PLM_001_20230824T070101_001053>
<Item id=EMIT_L2B_CH4PLM_001_20230824T070101_001052>
<Item id=EMIT_L2B_CH4PLM_001_20230824T070101_001051>
<Item id=EMIT_L2B_CH4PLM_001_20230824T070101_001050>
<Item id=EMIT_L2B_CH4PLM_001_20230824T070101_001049>
<Item id=EMIT_L2B_CH

In [6]:
# Examine the first item in the collection
# Keep in mind that a list starts from 0, 1, 2... therefore items[0] is referring to the first item in the list/collection
search_items = search.item_collection()
search_items[0]

<Item id=EMIT_L2B_CH4PLM_001_20230825T170609_001111>

Note from the list of items that there can be multiple plumes detected at the same datetime:

In [7]:
for item in search_items:
    print(item.properties["datetime"])

2023-08-25T17:06:09Z
2023-08-25T17:06:09+00:00
2023-08-25T17:05:57+00:00
2023-08-25T07:50:05+00:00
2023-08-25T07:50:05+00:00
2023-08-25T07:47:43+00:00
2023-08-25T06:13:13+00:00
2023-08-24T17:54:01+00:00
2023-08-24T17:53:37+00:00
2023-08-24T17:53:37+00:00
2023-08-24T17:53:37+00:00
2023-08-24T08:39:31+00:00
2023-08-24T08:39:07+00:00
2023-08-24T07:01:01+00:00
2023-08-24T07:01:01+00:00
2023-08-24T07:01:01+00:00
2023-08-24T07:01:01+00:00
2023-08-24T07:01:01+00:00
2023-08-24T07:01:01+00:00
2023-08-24T07:00:49+00:00
2023-08-24T07:00:49+00:00
2023-08-24T07:00:49+00:00
2023-08-24T07:00:49+00:00
2023-08-24T07:00:37+00:00


For this reason, we will use the item `id` to reference the items specifically. Let's make a dictionary where we can do this more efficiently:

In [8]:
items_dict = {item.id[20::]: item for item in search_items}
# Print the keys of our dictionary, which we'll use to reference specific plumes
print(items_dict.keys())

dict_keys(['20230825T170609_001111', '20230825T170609_001109', '20230825T170557_001108', '20230825T075005_001115', '20230825T075005_001114', '20230825T074743_001112', '20230825T061313_001116', '20230824T175401_001039', '20230824T175337_001038', '20230824T175337_001037', '20230824T175337_001036', '20230824T083931_001044', '20230824T083907_001045', '20230824T070101_001053', '20230824T070101_001052', '20230824T070101_001051', '20230824T070101_001050', '20230824T070101_001049', '20230824T070101_001048', '20230824T070049_001193', '20230824T070049_001059', '20230824T070049_001058', '20230824T070049_001057', '20230824T070037_001061'])


In [9]:
# Before we go further, let's pick which asset to focus on for the remainder of the notebook. 
# This dataset only has one asset to choose from:
asset_name = "ch4-plume-emissions"

# Creating Maps using Folium
You will now explore global methane emission plumes from point sources and visualize the results on a map using `folium`.

## Fetch Imagery from Raster API
Here we get information from the `Raster API` which we will add to our map in the next section.

In [10]:
# You can change the <datetime_plumenumber> key below to look at a different plume.
observation_date_1 = items_dict["20230825T170609_001109"]

# Extract collection name and item ID
collection_id = observation_date_1.collection_id
item_id = observation_date_1.id

In [11]:
object = observation_date_1.assets[asset_name]
raster_bands = object.extra_fields.get("raster:bands", [{}])
rescale_values = {
    "max": raster_bands[0].get("histogram", {}).get("max"),
    "min": 0,
}

print(rescale_values)
print(raster_bands)

{'max': 3096.197021484375, 'min': 0}
[{'scale': 1.0, 'nodata': -9999.0, 'offset': 0.0, 'sampling': 'area', 'data_type': 'float32', 'histogram': {'max': 3096.197021484375, 'min': -2284.946044921875, 'count': 11, 'buckets': [2, 86, 937, 4291, 6402, 3583, 859, 159, 16, 8]}, 'statistics': {'mean': 116.27998225539987, 'stddev': 537.0677841963014, 'maximum': 3096.197021484375, 'minimum': -2284.946044921875, 'valid_percent': 66.29751328546509}}]


Now, you will pass the `item id`, `collection name`, `asset name`, and the `rescale values` to the Raster API endpoint, along with a colormap. This step tells the Raster API which collection, item, and asset you want to view, specifying the colormap and colorbar ranges to use for visualization. The API returns a JSON with information about the requested image. Each image will be referred to as a tile.

In [12]:
# Choose a colormap for displaying the tiles
# Make sure that the capitalization matches Matplotlib standards
# For more information on Colormaps in Matplotlib, please visit https://matplotlib.org/stable/users/explain/colors/colormaps.html
color_map = "plasma"

In [13]:
# Make a GET request to retrieve information for the date specified
observation_date_1_tile = requests.get(
    f"{RASTER_API_URL}/collections/{collection_id}/items/{item_id}/tilejson.json?"
    f"&assets={asset_name}"
    f"&color_formula=gamma+r+1.05&colormap_name={color_map.lower()}"
    f"&rescale=0,1500"
).json()

# Print the properties of the retrieved granule to the console
observation_date_1_tile


{'tilejson': '2.2.0',
 'version': '1.0.0',
 'scheme': 'xyz',
 'tiles': ['https://earth.gov/ghgcenter/api/raster/collections/emit-ch4plume-v1/items/EMIT_L2B_CH4PLM_001_20230825T170609_001109/tiles/WebMercatorQuad/{z}/{x}/{y}@1x?assets=ch4-plume-emissions&color_formula=gamma+r+1.05&colormap_name=plasma&rescale=0%2C1500'],
 'minzoom': 0,
 'maxzoom': 24,
 'bounds': [-96.8337863674636,
  32.646954057997895,
  -96.69877046991976,
  32.70063507750328],
 'center': [-96.76627841869168, 32.67379456775059, 0]}

In [15]:
# Set initial zoom and center of map for plume Layer
# We'll use the "center" variable from our loaded tile to set the center of the map
# Note that we specify "tiles=None" because in the next step we're going to set a custom tile to serve as our underlying world map.
map_ = folium.Map(location=(observation_date_1_tile["center"][1], observation_date_1_tile["center"][0]), zoom_start=12, tiles=None, tooltip = 'test tool tip')
# Specify a custom imagery source for the underlying map
folium.TileLayer(tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}.png', name='ESRI World Imagery', attr='Tiles &copy; Esri &mdash; Source: Esri, i-cubed, USDA, USGS, AEX, GeoEye, Getmapping, Aerogrid, IGN, IGP, UPR-EGP, and the GIS User Community',overlay='True').add_to(map_)
# Add place labels on top
folium.TileLayer(tiles='https://server.arcgisonline.com/arcgis/rest/services/Reference/World_Boundaries_and_Places/MapServer/tile/{z}/{y}/{x}.png',name='ESRI World Boundaries and Places',attr='Tiles &copy; Esri &mdash; Source: Esri, i-cubed, USDA, USGS, AEX, GeoEye, Getmapping, Aerogrid, IGN, IGP, UPR-EGP, and the GIS User Community',overlay='True').add_to(map_)

# Use the 'TileLayer' library display the raster layer
map_layer = TileLayer(
    tiles=observation_date_1_tile["tiles"][0], # Path to retrieve the tile
    name=f'{items[0].assets[asset_name].title}', # Give this layer a title
    overlay='True', # The layer can be overlaid on the map
    attr="GHG", # Set the attribution
    opacity=1, # Adjust the transparency of the layer
)
map_layer.add_to(map_)

# Adjust map elements 
folium.LayerControl(collapsed=False, position='topright').add_to(map_)

# Add colorbar
# We can use one of 'generate_html_colorbar' from the 'ghgc_utils' module 
# to create an HTML colorbar representation.
legend_html = ghgc_utils.generate_html_colorbar(
                color_map,
                rescale_values,
                label='Methane Concentration (ppm-m)'
    )

# Add colorbar to the map
map_.get_root().html.add_child(folium.Element(legend_html))


# Visualizing the map
map_

## Summary

In this notebook we have successfully completed the following steps for the STAC collection for the EMIT Methane Point Source Plume Complexes dataset:
1.  Install and import the necessary libraries
2.  Fetch the collection from STAC collections using the appropriate endpoints
3.  Count the number of existing granules within the collection
4.  Map the methane emission plumes

If you have any questions regarding this user notebook, please contact us using the [feedback form](https://docs.google.com/forms/d/e/1FAIpQLSeVWCrnca08Gt_qoWYjTo6gnj1BEGL4NCUC9VEiQnXA02gzVQ/viewform).